In [15]:
import sys

assert sys.version_info >= (3, 7)

In [16]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [17]:
from pathlib import Path

In [18]:
# define the default font sizes to make the figures prettier:

In [19]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="lecun_normal",
                                    activation="selu"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [20]:
dataset = tf.keras.datasets.cifar10.load_data()

In [21]:
optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=7e-4) #Legacy cause made on a m2
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [22]:
(X_train_full, y_train_full), (X_test, y_test) = dataset

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [23]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [24]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_SELU_le_Cun_model.h5",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [25]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10642), started 0:50:24 ago. (Use '!kill 10642' to kill it.)

In [26]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.9345 - accuracy: 0.3052 - val_loss: 1.8802 - val_accuracy: 0.3170
Epoch 2/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.7112 - accuracy: 0.3906 - val_loss: 1.7583 - val_accuracy: 0.3590
Epoch 3/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.6095 - accuracy: 0.4337 - val_loss: 1.6207 - val_accuracy: 0.4202
Epoch 4/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.5396 - accuracy: 0.4548 - val_loss: 1.6050 - val_accuracy: 0.4410
Epoch 5/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.4788 - accuracy: 0.4792 - val_loss: 1.6405 - val_accuracy: 0.4290
Epoch 6/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.4347 - accuracy: 0.4968 - val_loss: 1.5203 - val_accuracy: 0.4488
Epoch 7/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.3857 - accuracy: 0.5163 - val_loss: 1.5402 - val_ac

In [28]:
#Lets apply dropout to see if it regularizes better !